In [1]:
from skimage import io, color, transform
import numpy as np
import time
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
# Read data set
data_dir = '/Users/kunpenghao/Desktop/Study_in_SIT/EE_628/project'
test_str = data_dir + '/data/test1/*.jpg'
train_str = data_dir + '/data/train/*.jpg'

def convert_gray(f): 
    rgb = io.imread(f)
    rgb = transform.resize(rgb, (28, 28))
    return color.rgb2gray(rgb)

test_im = io.ImageCollection(test_str, load_func = convert_gray)
train_im = io.ImageCollection(train_str, load_func = convert_gray)

train = [0 for i in range(25000)]
test = [0 for i in range(12500)]

time_start = time.time()
for i in range(len(train_im)):
    pic = train_im[i]
    train[i] = pic
for i in range(len(test_im)):
    pic = test_im[i]
    test[i] = pic
time_end = time.time()
print(time_end - time_start)

train = np.array(train)
test = np.array(test)

train = train.reshape((len(train), np.prod(train.shape[1:])))
test = test.reshape((len(test), np.prod(test.shape[1:])))

print(train.shape)
print(test.shape)

/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


202.6029109954834
(25000, 784)
(12500, 784)


In [3]:
# Make class for train_set
# x_train(20000), 0-9999 cat; 10000-19999 dog
# x_val(5000), 0-2499 cat; 2500-4999 dog
# y_train(20000) y_val(5000), cat = 1, dog = 0
cat_train = train[:10000]
cat_val = train[10000:12500]
dog_train = train[12500:22500]
dog_val = train[22500:]
x_train = np.zeros([20000, 784])
x_val = np.zeros([5000, 784])
x_train[:10000] = cat_train
x_train[10000:] = dog_train
x_val[:2500] = cat_val
x_val[2500:] = dog_val
y_train = np.zeros([20000, 1])
y_val = np.zeros([5000, 1])
y_train[10000:] = 1
y_val[2500:] = 1

#y_train = y_train.tolist()
#y_val = y_val.tolist()
#x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
#x_val = x_val.reshape(x_val.shape[0], 28, 28, 1)
#x_test = test.reshape(test.shape[0], 28, 28, 1)

In [4]:
from keras.utils import np_utils
number_of_classes = 2

Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_val = np_utils.to_categorical(y_val, number_of_classes)

y_train[0], Y_train[0]

Using TensorFlow backend.


(array([0.]), array([1., 0.], dtype=float32))

In [5]:
from keras.layers import Input, Dense, Lambda
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization

model = Sequential()

model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.1))
model.add(Dense(128))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dense(32))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.2))
model.add(Dense(2))
model.add(Activation('softmax'))

In [6]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.fit(x_train, Y_train,
                epochs=50,
                batch_size=512,
                shuffle=True,
                validation_data=(x_val, Y_val))

Train on 20000 samples, validate on 5000 samples
Epoch 1/50
20000/20000 [==============================] - 2s 122us/step - loss: 0.6986 - acc: 0.5255 - val_loss: 0.6796 - val_acc: 0.5660
Epoch 2/50
20000/20000 [==============================] - 1s 60us/step - loss: 0.6846 - acc: 0.5522 - val_loss: 0.6711 - val_acc: 0.5882
Epoch 3/50
20000/20000 [==============================] - 1s 61us/step - loss: 0.6753 - acc: 0.5752 - val_loss: 0.6718 - val_acc: 0.5806
Epoch 4/50
20000/20000 [==============================] - 1s 62us/step - loss: 0.6749 - acc: 0.5725 - val_loss: 0.6610 - val_acc: 0.5952
Epoch 5/50
20000/20000 [==============================] - 1s 62us/step - loss: 0.6653 - acc: 0.5984 - val_loss: 0.6655 - val_acc: 0.5978
Epoch 6/50
20000/20000 [==============================] - 1s 61us/step - loss: 0.6635 - acc: 0.5993 - val_loss: 0.6597 - val_acc: 0.6048
Epoch 7/50
20000/20000 [==============================] - 1s 62us/step - loss: 0.6589 - acc: 0.6090 - val_loss: 0.6589 - val_acc

In [7]:
score = model.evaluate(x_val, Y_val)
print('Test accuracy: ', score[1])

5000/5000 [==============================] - 0s 47us/step
Test accuracy:  0.6386


In [10]:
import pandas as pd
predictions = model.predict_classes(test)
predictions = list(predictions)
pic = []
for i in range(12500):
    pic.append(i+1)

sub = pd.DataFrame({'id': pic,'label': predictions})
sub.to_csv('./FullyConnected_output.csv', index=False)